# Convulational Neural Network Assignment
The goal of this project is to train a regression CNN to predict the resolution. We
will use the square loss functions on the training examples $(\mathbf{x}_i, y_i),i=1, \dots,n$:

$$S(\mathbf{w})=\frac{1}{n}\Sigma_{i=1}^n(y_i-f_\mathbf{w}(\mathbf{x}_i))^2 + \lambda ||\mathbf{w}||^2$$

Besides the loss function, we will measure the $R^2$

$R^2=1-\frac{\Sigma_{i=1}^n(y_i-\hat{y}_i)^2}{\Sigma_{i=1}^n(y_i-\bar{y}_i)^2}$

where $\hat{y}=f_\mathbf{w}(\mathbf{x}_i)$ and $\bar{y}=\frac{1}{n}\Sigma_{i=1}^n y_i$.

Experiment with different CNN architectures to obtain a good result. One example of a CNN you could use contains five convolutional layers with stride $1$ and zero
padding, the first four with filters of size $5 \times 5$ with or without holes (atrous), and the
last of the appropriate size to obtain a $1 \times 1$ output. The first two convolutions have
$16$ filters, the next two have $32$ filters, and the last has one filter. The first three convolutions are followed by $2 \times 2$ max pooling with stride $2$ respectively. The fourth
convolution layer is followed by ReLU.


#### Import dependencies

In [33]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
from scipy.special import expit
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.losses import MeanSquaredError
import os

#### Data loading function

In [34]:

def get_data():
    data_path = 'data/cnn'
    train_path = os.path.join(data_path,'cnntrain')
    test_path = os.path.join(data_path,'cnntest')

    data_gen_train = ImageDataGenerator()
    train_dataframe = pd.DataFrame(
        zip([os.path.join(train_path, image) for image in os.listdir(train_path)],
        [int(filename[:2])/100 for filename in os.listdir(train_path)]),
        columns=['path', 'resolution'])
    image_gen_train= data_gen_train.flow_from_dataframe(train_dataframe,
        x_col='path', y_col='resolution', class_mode='raw', target_size=(64,64))

    data_gen_test = ImageDataGenerator()
    test_dataframe = pd.DataFrame(
        zip([os.path.join(test_path, image) for image in os.listdir(test_path)],
        [int(filename[:2])/100 for filename in os.listdir(test_path)]),
        columns=['path', 'resolution'])
    image_gen_test= data_gen_test.flow_from_dataframe(test_dataframe,
        x_col='path', y_col='resolution', class_mode='raw', target_size=(64,64))

    def normalized_image_gen(image_gen):
        while True:
            batch, label = next(image_gen)
            yield batch / 255, label
    return normalized_image_gen(image_gen_train), normalized_image_gen(image_gen_test), train_dataframe.shape[0] // 32, test_dataframe.shape[0]

#### Loss function

In [35]:
def loss(x, y, w, f_w, lmda):
    return np.mean(y - f_w(x) + lmda*np.linalg.norm(w)**2)

#### $R^2$ function

In [36]:
def R2(y_true, y_predicted):
    return 1 - np.sum((y_true-y_predicted)**2) / np.sum((y_true-np.mean(y_true))**2)

#### Part A
Train a CNN for $100$ epochs with momentum $0.9$ using the square loss (1). Use
the SGD optimizer with an appropriate learning rate and $\lambda = 0.0001$ (weight
decay). Start with minibatch size $32$ and double it every $20$ epochs and to obtain
a good training $R^2$
(at least $0.9$). Show a plot of the loss function vs epoch
number for the training set and the test set. Show another plot of the training and
test $R^2$ vs epoch number. (4 points)

In [38]:
def build_cnn(train):
    images, labels = next(train)
    flattened_images = np.reshape(images, (images.shape[0], 4096*3))
    model = Sequential()
    model.add(Dense(8, input_dim=flattened_images.shape[1], activation="relu"))
    model.add(Dense(4, activation="relu"))
    model.add(Dense(1, activation="linear"))
    model.compile(optimizer="SGD", loss=MeanSquaredError())
    return model

def train_cnn(model, train, test, train_steps, test_steps):
    def my_gen(gen):
        while True:
            images, labels = next(gen)
            images=  np.reshape(images, (images.shape[0], 4096*3))
            yield images, labels
    model.fit(my_gen(train), steps_per_epoch=train_steps)
    print(model.evaluate(my_gen(test), steps=test_steps, use_multiprocessing=False))

train, test, train_steps, test_steps = get_data()
train_steps, test_steps = 50,50 # only train on some of the data to make testing faster
print("train_steps", train_steps)
print("test_steps", test_steps)
model= build_cnn(train)
train_cnn(model, train, test, train_steps, test_steps)

Found 18059 validated image filenames.
Found 2261 validated image filenames.
train_steps 50
test_steps 50
Epoch 1/1
50/50 [==============================] - 1s 11ms/step
0.1392807960510254
